In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
import keras.models
#from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D ,Flatten , Dense, Dropout
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from PIL import Image
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D , BatchNormalization;
#from models import custom_convnet
from keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import classification_report

C:\Users\nehaw\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
job=pd.read_csv(r"C:\Users\vikas\Desktop\csv_files\fake_job_postings.csv", encoding='latin1')

In [3]:
# df2=job[job['benefits'].isna()]

In [4]:
# df2.fraudulent.value_counts()

0    6846
1     364
Name: fraudulent, dtype: int64

In [31]:
job.location.value_counts()

GB, LND, London          718
US, NY, New York         658
US, CA, San Francisco    472
GR, I, Athens            464
US, ,                    339
                        ... 
GB, SFK, Leiston           1
GB, LND, Hammersmith       1
US, WA, Seattle            1
BE                         1
GB, WSX, Chichester        1
Name: location, Length: 3105, dtype: int64

In [5]:
job['x'] = job['salary_range'].apply(str)+' '+job['company_profile'].apply(str)+' '+job['has_company_logo'].apply(str)+' '+job['location'].apply(str)+' '+job['benefits'].apply(str)

In [6]:
job = job.loc[: , ['x','fraudulent']]

In [9]:
train,test = train_test_split(job)

In [10]:
df_temp = train[train.fraudulent==1]
train = pd.concat([train,df_temp,df_temp,df_temp])

In [11]:
x_train=train.x
y_train = train.loc[:,['fraudulent']]

x_test=test.x
y_test = test.loc[:,['fraudulent']]

In [8]:
from sklearn.model_selection import train_test_split
# x_train , x_test , y_train, y_test = train_test_split(job['x'] , job['fraudulent'] , test_size = .2)

In [266]:
# # Assuming data is your dataset
# from sklearn.model_selection import train_test_split

# # Adjust validation_split as needed
# X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True)


In [12]:
y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

In [13]:
max_num_words=10000             ## from the entire corpus select 10000 words
seq_len=50                      ## how many words out of 10000 you wish to take from 1 doc
embedding_size=100

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [16]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(job.x)
x_train = tokenizer.texts_to_sequences(x_train)

x_test = tokenizer.texts_to_sequences(x_test)

In [17]:
x_train = pad_sequences(x_train , maxlen = seq_len)

x_test = pad_sequences(x_test , maxlen = seq_len)

In [18]:
x_test.shape

(4470, 50)

In [19]:
model = Sequential()
model.add(Embedding(input_dim = max_num_words,
                   input_length = seq_len,
                   output_dim = embedding_size))

C:\Users\nehaw\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from keras.layers import Bidirectional

In [21]:
model.add(Bidirectional(LSTM(64)))
model.add(Dense(2, activation = 'softmax'))  #o/p layer
adam = Adam(learning_rate = 0.001)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [22]:
model.fit(x_train,y_train,batch_size=32,epochs =5)

Epoch 1/5
481/481 ━━━━━━━━━━━━━━━━━━━━ 42s 67ms/step - accuracy: 0.8864 - loss: 0.2735
Epoch 2/5
481/481 ━━━━━━━━━━━━━━━━━━━━ 32s 67ms/step - accuracy: 0.9684 - loss: 0.0671
Epoch 3/5
481/481 ━━━━━━━━━━━━━━━━━━━━ 30s 62ms/step - accuracy: 0.9778 - loss: 0.0521
Epoch 4/5
481/481 ━━━━━━━━━━━━━━━━━━━━ 41s 62ms/step - accuracy: 0.9810 - loss: 0.0438
Epoch 5/5
481/481 ━━━━━━━━━━━━━━━━━━━━ 30s 63ms/step - accuracy: 0.9816 - loss: 0.0422


In [23]:
pred = model.predict(x_test)
pred

140/140 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step


array([[9.9999511e-01, 4.8599063e-06],
       [9.9953640e-01, 4.6365321e-04],
       [9.9993074e-01, 6.9311929e-05],
       ...,
       [9.9993610e-01, 6.3870233e-05],
       [9.9952936e-01, 4.7071362e-04],
       [9.9987900e-01, 1.2093656e-04]], dtype=float32)

In [24]:
pred_classes= pred.argmax(axis = 1)
#y_test= y_test.argmax(axis = 1)

In [25]:
tab = confusion_matrix(y_test , pred_classes)
tab

array([[4151,  104],
       [  74,  141]], dtype=int64)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score


In [27]:
accuracy_score(y_test , pred_classes)*100

96.01789709172259

In [28]:
print(classification_report(y_test,pred_classes))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4255
           1       0.58      0.66      0.61       215

    accuracy                           0.96      4470
   macro avg       0.78      0.82      0.80      4470
weighted avg       0.96      0.96      0.96      4470



In [283]:
model.save('fake_job_LSTM.h5')

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
